Run only if you have saved relevant json data to disk. See main.ipynb

In [1]:
import os
import re
import json
import pickle
import pandas as pd
from pprint import pp

In [2]:
dirs={}
dirs['project']=os.path.abspath('..')+'\\'
dirs['code']=dirs['project']+'code\\'
dirs['data']=dirs['project']+'data\\'
dirs['models']=dirs['project']+'models\\'
dirs['results']=dirs['project']+'results\\'
dirs['rawresults']=dirs['results']+'raw\\'

In [3]:
with open(dirs['data']+'facets.pkl', 'rb') as f:
    facets=pickle.load(f)
facets.simple = {}
for facet in [f['name'] for f in facets.json().get('facets')]:
    facets.simple[facet] = {p['value']: {'raw': p['rawValue'], 'count': p['count']} for p in [f for f in facets.json().get('facets') if f['name']==facet][0]['values']}

I'm just gonna go and load all the data now. Probably not the smartest thing to do - at least do it per framework programme - but it will suffice for now.

In [4]:
jsons={}
# for programme in facets.simple['frameworkProgramme'].keys():
# for programme in ['Digital Europe Programme (DIGITAL)']:
# for programme in ['Horizon Europe (HORIZON)']:
for programme in ['Horizon Europe (HORIZON)', 'Digital Europe Programme (DIGITAL)']:
    raw=facets.simple['frameworkProgramme'][programme]['raw']
    jsons[programme]={}
    for j in [f for f in os.listdir(dirs['data']+raw) if f.endswith('.json')]:
        with open(dirs['data']+raw+'\\'+j, 'r') as f:
            jsons[programme][j[:-5]]=json.load(f)

In [5]:
# jsons['Digital Europe Programme (DIGITAL)']['DIGITAL-2021-CLOUD-AI-01-DATA-TECH']

In [6]:
transformdict = {
    'HORIZON-CL4': {
        'TWIN-TRANSITION': ('Industry', 'D1. TWIN-TRANSITION', '1. Climate neutral, circular and digitised production'),
        'RESILIENCE': ('Industry', 'D2. RESILIENCE', '2. Increased autonomy in key strategic value chains for resilient industry'),
        'DATA': ('Digital', 'D3. DATA', '3. World leading data and computing technologies'),
        'DIGITAL-EMERGING': ('Digital', 'D4. DIGITAL-EMERGING', '4. Digital and emerging technologies for competitiveness and fit for the Green Deal'),
        'SPACE': ('Space', 'D5. SPACE', '5. Open strategic autonomy in developing, deploying and using global space-based infrastructures, services, applications and data'),
        'HUMAN': ('Digital', 'D6. HUMAN', '6. A human-centred and ethical development of digital and industrial technologies'),
        'other': ('Digital', 'HE & DEP Other', 'Other actions not subject to calls for proposals & Programme Support Actions')
    },
    'HORIZON-EUROHPC-JU': {
        # 'SO1: HPC': (), 
        'other': ('Digital', 'SO1: HPC', 'High Performance Computing', 'EuroHPC') # I'm just gonna chuck this into the same bucket as the DIGITAL EuroHPC topics, for ease, for now.
    },
    'HORIZON-JU-Chips': {
        'other': ('Digital', 'HE & DEP Other', 'Other actions not subject to calls for proposals & Programme Support Actions', 'Chips')
    },
    'HORIZON-JU-SNS': {
        'other': ('Digital', 'HE & DEP Other', 'Other actions not subject to calls for proposals & Programme Support Actions', 'SNS')
    },
    'HORIZON-KDT-JU': {
        'other': ('Digital', 'HE & DEP Other', 'Other actions not subject to calls for proposals & Programme Support Actions', 'KDT')
    },
    'HORIZON-Chips': {
        'other': ('Digital', 'HE & DEP Other', 'Other actions not subject to calls for proposals & Programme Support Actions', 'Chips')
    },
    'HORIZON-EUSPA': {
        'SPACE': ('Space', 'D5. SPACE', '5. Open strategic autonomy in developing, deploying and using global space-based infrastructures, services, applications and data')
    },
    'DIGITAL': {
        'ADVANCED-DIGITAL': ('Digital', 'SO4: SKILLS', 'Advanced Digital Skills'), #assumption WL
        'ADVANCED-SKILLS': ('Digital', 'SO4: SKILLS', 'Advanced Digital Skills'), #assumption WL
        'AI': ('Digital', 'SO2: CLOUD-DATA-AI', 'Cloud, Data and Artificial Intelligence'), #assumption WL
        'AI-ACT': ('Digital', 'SO2: CLOUD-DATA-AI', 'Cloud, Data and Artificial Intelligence'), #assumption WL
        'BESTUSE': ('Digital', 'SO5: DEPLOY', 'Accelerating Best Use of Technologies'), #assumption WL
        'BESTUSE-TECH': ('Digital', 'SO5: DEPLOY', 'Accelerating Best Use of Technologies'), #assumption WL
        'CLOUD-AI': ('Digital', 'SO2: CLOUD-DATA-AI', 'Cloud, Data and Artificial Intelligence'),
        'CLOUD-AI-B': ('Digital', 'SO2: CLOUD-DATA-AI', 'Cloud, Data and Artificial Intelligence'),
        'CLOUD-DATA': ('Digital', 'SO2: CLOUD-DATA-AI', 'Cloud, Data and Artificial Intelligence'),
        'CLOUD-DATA-AI': ('Digital', 'SO2: CLOUD-DATA-AI', 'Cloud, Data and Artificial Intelligence'),
        'CULTURAL': ('Digital', 'SO2: CLOUD-DATA-AI', 'Cloud, Data and Artificial Intelligence'),
        'CYBER': ('Digital', 'SO3: CYBER', 'Cybersecurity'),
        'DATA-SEC-LAW': ('Digital', 'SO2: CLOUD-DATA-AI', 'Cloud, Data and Artificial Intelligence'), #assumption WL
        'DEPLOY': ('Digital', 'SO5: DEPLOY', 'Accelerating Best Use of Technologies'),
        'DEPLOY-BESTUSE-TECH': ('Digital', 'SO5: DEPLOY', 'Accelerating Best Use of Technologies'),
        'EDIH': ('Digital', 'SO2: CLOUD-DATA-AI', 'Cloud, Data and Artificial Intelligence'),
        'EDIH-INITIAL': ('Digital', 'SO2: CLOUD-DATA-AI', 'Cloud, Data and Artificial Intelligence'),
        'GOVTECH': ('Digital', 'SO5: DEPLOY', 'Accelerating Best Use of Technologies'),
        'PREPACTS-DS': ('Digital', 'SO2: CLOUD-DATA-AI', 'Cloud, Data and Artificial Intelligence'),
        # 'PROGRAM-SUPPORT': ('Digital', 'HE & DEP Other', 'Other actions not subject to calls for proposals & Programme Support Actions'),    # other
        'QCI': ('Digital', 'SO3: CYBER', 'Cybersecurity'),
        'SKILLS': ('Digital', 'SO4: SKILLS', 'Advanced Digital Skills'),
        'TRAINING': ('Digital', 'SO4: SKILLS', 'Advanced Digital Skills'),
        'TRUST': ('Digital', 'SO5: DEPLOY', 'Accelerating Best Use of Technologies'),
        'other': ('Digital', 'HE & DEP Other', 'Other actions not subject to calls for proposals & Programme Support Actions')
    },
    'DIGITAL-Chips': {
        # 'CSA-CCC': (), #?
        # 'CSA-CDP': (), #?
        # 'SG-CCC': (), #?
        # 'SG-CPL': (), #?
        'other': ('Digital', 'HE & DEP Other', 'Other actions not subject to calls for proposals & Programme Support Actions', 'Chips')
    },
    'DIGITAL-ECCC': {
        'CYBER': ('Digital', 'SO3: CYBER', 'Cybersecurity'), #assumption WL
        'CYBER-B': ('Digital', 'SO3: CYBER', 'Cybersecurity'), #assumption WL
        'DEPLOY-CYBER': ('Digital', 'SO3: CYBER', 'Cybersecurity'), #assumption WL
        'DEPLOY-NCC': ('Digital', 'SO3: CYBER', 'Cybersecurity'), #assumption WL
        'other': ('Digital', 'HE & DEP Other', 'Other actions not subject to calls for proposals & Programme Support Actions')
    },
    'DIGITAL-EUROHPC-JU': {
        'ACADEMY': ('Digital', 'SO1: HPC', 'High Performance Computing', 'EuroHPC'), #assumption WL
        'AISC': ('Digital', 'SO1: HPC', 'High Performance Computing', 'EuroHPC'), #assumption WL
        'APPSUPPORT': ('Digital', 'SO1: HPC', 'High Performance Computing', 'EuroHPC'), #assumption WL
        'NCC': ('Digital', 'SO1: HPC', 'High Performance Computing', 'EuroHPC'), #assumption WL
        'SME': ('Digital', 'SO1: HPC', 'High Performance Computing', 'EuroHPC'), #assumption WL
        'TRAINING': ('Digital', 'SO1: HPC', 'High Performance Computing', 'EuroHPC'), #assumption WL
        'other': ('Digital', 'HE & DEP Other', 'Other actions not subject to calls for proposals & Programme Support Actions', 'EuroHPC')
    },
    'DIGITAL-JU-Chips': {
        # 'SG-CPL': (), #?
        # 'other': ('Digital', 'JUs: Chips', 'Joint Undertaking: Chips', 'Chips')
        'other': ('Digital', 'HE & DEP Other', 'Other actions not subject to calls for proposals & Programme Support Actions', 'Chips')
    }
}

In [7]:
def get_destination_info(j, transformdict=transformdict):
    identifier = j['TopicDetails']['identifier']

    destdict = {}
    destdict['destination'] = re.sub(r' \(20[0-9]{2}[^a-zA-Z]*?$', '', j['TopicDetails'].get('destinationDescription', ''))
    m = re.search(r'-[0-9]{4}-', identifier)
    if m:
        destdict['programme'] = identifier[:m.span()[0]]
        destdict['year'] = identifier[m.span()[0]+1:m.span()[1]-1]
        destdict['topic'] = identifier[m.span()[1]:]
        n = re.search(r'^[0-9]{2}', destdict['topic']) # if topic identifier post-year starts with two numbers, that's the spec, not the cat. Assume no cat. 
        if n:
            destdict['topic_spec'] = destdict['topic']
        else:   # else split cat and spec.
            n = re.search(r'-[0-9]', destdict['topic'])
            if n:   # if we find a number indicator and the 'topic' does not start with a number
                destdict['topic_cat_orig'] = destdict['topic_cat'] = destdict['topic'][:n.span()[0]]
                destdict['topic_spec'] = destdict['topic'][n.span()[0]+1:]  # in theory this includes call number and topic number, or equivalent.

                if destdict['programme'] in transformdict.keys():
                    if destdict['topic_cat'] in transformdict[destdict['programme']].keys():
                        if len(transformdict[destdict['programme']][destdict['topic_cat']]) == 4:
                            destdict['domain'], destdict['topic_cat'], destdict['destination'], destdict['JU'] = transformdict[destdict['programme']][destdict['topic_cat']]
                        else:
                            destdict['domain'], destdict['topic_cat'], destdict['destination'] = transformdict[destdict['programme']][destdict['topic_cat']]
                    else:
                        if len(transformdict[destdict['programme']]['other']) == 4:
                            destdict['domain'], destdict['topic_cat'], destdict['destination'], destdict['JU'] = transformdict[destdict['programme']]['other']
                        else:
                            destdict['domain'], destdict['topic_cat'], destdict['destination'] = transformdict[destdict['programme']]['other']

    return destdict

In [8]:
def get_budget_info(j):

    budgetdict = j['TopicDetails']['budgetOverviewJSONItem']['budgetTopicActionMap']

    items = []
    for i in budgetdict.values():
        items.extend(i)

    budgetdict = [i for i in items if i['action'].startswith(k+' ')]
    budgetdict = [{k:v for k, v in d.items() if k != 'action'} for d in budgetdict] # remove action
    budgetdict = [i for n, i in enumerate(budgetdict) if i not in budgetdict[n + 1:]] # remove duplicate entities - these are just the way they combine multiple actions as rowindexes
    # if not [budgetdict[0]]*len(budgetdict) == budgetdict:   # if entries are not identical, warn - these are different actions belonging to the same topic, we sum their budgets later
    #     print('conflicting budget info at '+k)

    budgetdictyearcolumns = []
    for i in budgetdict:
        budgetdictyearcolumns.extend([k for k in i['budgetYearMap'].keys()])
    budgetdictyearcolumns = list(set(budgetdictyearcolumns))
    # if len(list(set(budgetdictyearcolumns)))>1: # if one action has multiple years, we sum the budgets to a total later
    #     print('multiple budget year columns for '+k)

    budgettotal = []
    expectedgrants = 0
    for d in budgetdict:
        budgettotal.extend(d.get('budgetYearMap').values())
        expectedgrants += d.get('expectedGrants', 0)
    budgettotal = sum(budgettotal)
    if expectedgrants == 0:
        expectedgrants = ''

    return budgettotal, expectedgrants

In [9]:
sorted(list(set([(get_destination_info(j).get('programme', 'NONE'), get_destination_info(j).get('topic_cat', 'NONE')) for k,j in jsons['Horizon Europe (HORIZON)'].items()])))

[('ERC', 'NONE'),
 ('ERC', 'PERA'),
 ('ERC', 'SJI'),
 ('HOIRZON-EIC', 'NONE'),
 ('HORIZON-CL2', 'DEMOCRACY'),
 ('HORIZON-CL2', 'HERITAGE'),
 ('HORIZON-CL2', 'HERITAGE-ECCCH'),
 ('HORIZON-CL2', 'TRANSFORMATIONS'),
 ('HORIZON-CL3', 'BM'),
 ('HORIZON-CL3', 'CS'),
 ('HORIZON-CL3', 'DRS'),
 ('HORIZON-CL3', 'FCT'),
 ('HORIZON-CL3', 'INFRA'),
 ('HORIZON-CL3', 'SSRI'),
 ('HORIZON-CL4', 'D1. TWIN-TRANSITION'),
 ('HORIZON-CL4', 'D2. RESILIENCE'),
 ('HORIZON-CL4', 'D3. DATA'),
 ('HORIZON-CL4', 'D4. DIGITAL-EMERGING'),
 ('HORIZON-CL4', 'D5. SPACE'),
 ('HORIZON-CL4', 'D6. HUMAN'),
 ('HORIZON-CL5', 'D1'),
 ('HORIZON-CL5', 'D2'),
 ('HORIZON-CL5', 'D3'),
 ('HORIZON-CL5', 'D4'),
 ('HORIZON-CL5', 'D5'),
 ('HORIZON-CL5', 'D6'),
 ('HORIZON-CL6', 'BIODIV'),
 ('HORIZON-CL6', 'CIRCBIO'),
 ('HORIZON-CL6', 'CLIMATE'),
 ('HORIZON-CL6', 'COMMUNITIES'),
 ('HORIZON-CL6', 'CircBio'),
 ('HORIZON-CL6', 'FARM2FORK'),
 ('HORIZON-CL6', 'GOVERNANCE'),
 ('HORIZON-CL6', 'ZEROPOLLUTION'),
 ('HORIZON-Chips', 'HE & DEP Other'

In [10]:
lookuptable={}
# for programme in facets.simple['frameworkProgramme'].keys():
# for programme in ['Digital Europe Programme (DIGITAL)']:
# for programme in ['Horizon Europe (HORIZON)']:
for programme in ['Horizon Europe (HORIZON)', 'Digital Europe Programme (DIGITAL)']:
    for k,j in jsons[programme].items():

        budgettotal, expectedgrants = get_budget_info(j)
        destdict = get_destination_info(j)

        lookuptable[k]={
            'Year': destdict.get('year', ''), 
            'Domain': destdict.get('domain', ''), 
            'Programme': destdict.get('programme', ''), 
            'Category original': destdict.get('topic_cat_orig', destdict.get('topic', '')),
            'Destination / Category': destdict.get('topic_cat', destdict.get('topic', '')),
            'Destination / Category FULL': destdict.get('destination', ''), 
            'Available Budget': budgettotal, 
            'Expected Grants': expectedgrants,
            'Joint Undertaking': destdict.get('JU', ''), 
        }
        
df = pd.DataFrame.from_dict(lookuptable, orient='index')
df.index.name = 'Topic'
# df.to_excel(dirs['rawresults']+'\\destination_lookup_table.xlsx')
df.to_csv(dirs['rawresults']+'\\destination_lookup_table.csv')
df.to_pickle(dirs['rawresults']+'\\destination_lookup_table.pkl')

In [11]:
df

,Year,Domain,Programme,Category original,Destination / Category,Destination / Category FULL,Available Budget,Expected Grants,Joint Undertaking
Topic,,,,,,,,,
ERC-2020-PERA-2,2020,,ERC,PERA,PERA,,30000.0,,
ERC-2021-ADG,2021,,ERC,ADG,ADG,,626000000.0,,
ERC-2021-COG,2021,,ERC,COG,COG,,633000000.0,,
ERC-2021-STG,2021,,ERC,STG,STG,,619000000.0,,
ERC-2022-ADG,2022,,ERC,ADG,ADG,,471000000.0,,
...,...,...,...,...,...,...,...,...,...
DIGITAL-EUROHPC-JU-2024-NCC-02-01,2024,Digital,DIGITAL-EUROHPC-JU,NCC,SO1: HPC,High Performance Computing,5000000.0,1,EuroHPC
DIGITAL-JU-Chips-2023-SG-CPL-1,2023,Digital,DIGITAL-JU-Chips,SG-CPL,HE & DEP Other,Other actions not subject to calls for proposa...,40000000.0,3,Chips
DIGITAL-JU-Chips-2023-SG-CPL-2,2023,Digital,DIGITAL-JU-Chips,SG-CPL,HE & DEP Other,Other actions not subject to calls for proposa...,10000000.0,3,Chips
